In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import os

In [ ]:
stroke_registry_data_path = '/Users/jk1/OneDrive - unige.ch/stroke_research/geneva_stroke_unit_dataset/data/stroke_registry/post_hoc_modified/stroke_registry_post_hoc_modified.xlsx'
patient_selection = '/Users/jk1/temp/opsum_extration_output/high_frequency_data_patient_selection.csv'
out_dir = '/Users/jk1/temp/opsum_extration_output'

In [ ]:
patient_selection_df = pd.read_csv(patient_selection, dtype=str)
full_data_df = pd.read_excel(stroke_registry_data_path)

In [ ]:
full_data_df['patient_id'] = full_data_df['Case ID'].apply(lambda x: x[8:-4])
full_data_df['EDS_last_4_digits'] = full_data_df['Case ID'].apply(lambda x: x[-4:])
full_data_df['case_admission_id'] = full_data_df['patient_id'].astype(str) + full_data_df['EDS_last_4_digits'].astype(str) + '_' + pd.to_datetime(full_data_df['Arrival at hospital'], format='%Y%m%d').dt.strftime('%d%m%Y').astype(str)
patient_selection_df['case_admission_id'] = patient_selection_df['patient_id'].astype(str) + patient_selection_df['EDS_last_4_digits'].astype(str) + '_' + pd.to_datetime(patient_selection_df['Arrival at hospital'], format='%Y%m%d').dt.strftime('%d%m%Y').astype(str)

In [ ]:
selected_full_data_df = full_data_df[
    full_data_df['case_admission_id'].isin(patient_selection_df['case_admission_id'].tolist())]

selected_full_data_df['begin_date'] = pd.to_datetime(selected_full_data_df['Arrival at hospital'],
                                                     format='%Y%m%d').dt.strftime('%d.%m.%Y') + ' ' + selected_full_data_df['Arrival time']

In [ ]:
all_imaging_columns = [
    '1st brain imaging type',
'1st brain imaging date',
'1st brain imaging time',
'Door to image (min.)',
'1st brain imaging result',
'Acute perf. imaging type',
'Acute perf. imaging result',
'1st vascular imaging type',
'1st vascular imaging result'
]

In [ ]:
column =  '1st brain imaging type'
selected_full_data_df[column].isna().sum()

In [ ]:
selected_full_data_df[column].value_counts()

In [ ]:
selected_full_data_df['onset_datetime'] = pd.to_datetime(pd.to_datetime(selected_full_data_df['Onset date'], format='%Y%m%d').dt.strftime('%d-%m-%Y') \
                                        + ' ' + selected_full_data_df['Onset time'], format='%d-%m-%Y %H:%M')
selected_full_data_df['imaging_datetime'] = pd.to_datetime(pd.to_datetime(selected_full_data_df['1st brain imaging date'], format='%Y%m%d').dt.strftime('%d-%m-%Y') \
                                        + ' ' + selected_full_data_df['1st brain imaging time'], format='%d-%m-%Y %H:%M')

In [ ]:
selected_full_data_df['begin_date']

In [ ]:
selected_full_data_df['onset_to_imaging_min'] = (pd.to_datetime(selected_full_data_df['imaging_datetime'], format='%d-%m-%Y %H:%M:%S') - pd.to_datetime(selected_full_data_df['onset_datetime'], format='%d-%m-%Y %H:%M:%S')).dt.total_seconds() / 60
selected_full_data_df['door_to_image_min'] = (pd.to_datetime(selected_full_data_df['imaging_datetime'], format='%d-%m-%Y %H:%M:%S') - pd.to_datetime(selected_full_data_df['begin_date'], format='%d.%m.%Y %H:%M')).dt.total_seconds() / 60
selected_full_data_df['onset_to_admission_min'] = (pd.to_datetime(selected_full_data_df['begin_date'], format='%d.%m.%Y %H:%M') - pd.to_datetime(selected_full_data_df['onset_datetime'], format='%d-%m-%Y %H:%M:%S')).dt.total_seconds() / 60

In [ ]:
ax = selected_full_data_df.plot.scatter(x='onset_to_imaging_min', y='onset_to_admission_min', color='blue')
selected_full_data_df.plot.scatter(x='onset_to_imaging_min', y='door_to_image_min', color='red', ax=ax)
ax.set_xlim([0, 1880])
ax.set_ylim([0, 1880])
plt.show()


In [ ]:
selected_full_data_df['onset_to_imaging_min'].isna().sum()

In [ ]:
selected_full_data_df[['onset_datetime', 'imaging_datetime', 'onset_to_admission_min', 'onset_to_imaging_min', 'door_to_image_min', 'Door to image (min.)', '1st brain imaging type']]

**Impression:** Too many missing values in imaging data -> therefore use admission timings
- eventually imaging results could be used